In [ ]:
# default_exp core

# Core

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export 
import os

import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import MNIST, ImageFolder
from torchvision.transforms import ToTensor, Resize, Compose, RandomResizedCrop, Normalize
import pytorch_lightning as pl

import fastai.vision.augment
import fastai.vision.data
# from fastai.vision.data import ImageDataLoaders
# from fastai.vision.augment import Resize

In [ ]:
#export
class ImageFolderDataModule(pl.LightningDataModule):

    def __init__(self, data_dir, batch_size):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.transform = Compose([
            Resize(224),
            RandomResizedCrop(224),
            ToTensor(),
            Normalize((0.1307,), (0.3081,))
        ])

    def setup(self, stage=None):
        data_dir = self.data_dir
        transform = self.transform
        
        self.dls = fastai.vision.data.ImageDataLoaders.from_folder(data_dir, item_tfms=fastai.vision.augment.Resize(224))
        self.trainset = ImageFolder(os.path.join(data_dir, 'train'), transform)
        self.testset = ImageFolder(os.path.join(data_dir, 'valid'), transform)

    def train_dataloader(self):
        return DataLoader(self.trainset, batch_size=self.batch_size)

    def valid_dataloader(self):
        return DataLoader(self.testset, batch_size=self.batch_size)

In [ ]:
data_dir = 'Datasets/Affective_Image_Classification_Using_Features_inpired_by_Psychology_and_Art_Theory'
dm = ImageFolderDataModule(data_dir, 32)
dm.setup()

In [ ]:
for x,y in dm.train_dataloader():
    test_eq(type(x), torch.Tensor) 
    test_eq(type(y), torch.Tensor) 

In [ ]:
#export
class CNNModel(pl.LightningModule):
    def __init__(self, model=None, pretrained=False, log_level=10):
        super().__init__()
        
        assert model is not None, 'Select model from torchvision'
        self.model = eval(f'torchvision.models.{model}(pretrained={pretrained})')

    def forward(self, x):
        return torch.relu(self.model(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
trainer = pl.Trainer()
model = CNNModel('vgg11', pretrained=True)
trainer.fit(model, dm)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores
Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /home/atom/.cache/torch/hub/checkpoints/vgg11-bbd30ac9.pth

  | Name  | Type | Params
-------------------------------
0 | model | VGG  | 132 M 
-------------------------------
132 M     Trainable params
0         Non-trainable params
132 M     Total params


1